In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
# chemins (à adapter si besoin)
base = Path(".")
ds_poids = xr.open_dataset(base / "poids_regions_finie.nc")
df_villes = pd.read_parquet(base / "villes_mapping.parquet")

print(ds_poids)
print(df_villes.head())

# Nettoyage 


In [ ]:
lat_col = next(c for c in df_villes.columns if 'lat' in c.lower())
lon_col = next(c for c in df_villes.columns if 'lon' in c.lower())

col_ville = "Ville" if "Ville" in df_villes.columns else df_villes.columns[0]
col_cp = "CodePostal" if "CodePostal" in df_villes.columns else None

df_villes = df_villes.dropna(subset=[lat_col, lon_col])

def make_label(row):
    if col_cp:
        return f"{row[col_ville]} ({row[col_cp]})"
    return row[col_ville]

df_villes["label"] = df_villes.apply(make_label, axis=1)

df_villes = df_villes.rename(columns={lat_col: "lat", lon_col: "lon"})

weights = ds_poids["weights"]

lat_name = next(c for c in weights.coords if "lat" in c.lower())
lon_name = next(c for c in weights.coords if "lon" in c.lower())

print("Coords détectées :", lat_name, lon_name)
print("Régions disponibles :", ds_poids.region.values)



# Choix région

In [ ]:
region_test = ds_poids.region.values[0]  # ou "Occitanie", "Bretagne", etc.
print("Région testée :", region_test)

mask = weights.sel(region=region_test)

print("Mask min/max :", float(mask.min()), float(mask.max()))


In [ ]:
villes_ok = []
villes_ko = []

for _, row in df_villes.iterrows():
    val = mask.sel(
        **{
            lat_name: row["lat"],
            lon_name: row["lon"]
        },
        method="nearest"
    ).values

    if np.isfinite(val) and val > 0:
        villes_ok.append(row)
    else:
        villes_ko.append(row)

df_ok = pd.DataFrame(villes_ok)
df_ko = pd.DataFrame(villes_ko)

print("Villes retenues :", len(df_ok))
print("Villes rejetées :", len(df_ko))


In [ ]:
plt.figure(figsize=(8, 8))

# Masque de la région
mask.plot(
    cmap="Greys",
    alpha=0.4,
    add_colorbar=False
)

# Villes rejetées
plt.scatter(
    df_ko["lon"], df_ko["lat"],
    s=5, c="red", alpha=0.3, label="Hors région"
)

# Villes acceptées
plt.scatter(
    df_ok["lon"], df_ok["lat"],
    s=10, c="green", alpha=0.8, label="Dans la région"
)

plt.title(f"Test filtrage des villes — {region_test}")
plt.legend()
plt.show()
